In [2]:
import sys
import os

# 현재 Jupyter Notebook이 실행 중인 디렉토리를 기준으로 상위 경로 추가
sys.path.append(os.path.abspath(".."))

# 모듈 불러오기
from dart_tool.insider_trade_tool import DARTExecutiveShareholdingAPI, DARTMajorStockReportAPI

# 기업 코드

In [3]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
# API 키 입력
DART_API_KEY = os.getenv("DART_API_KEY")
# 고유번호 요청 URL
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={DART_API_KEY}"

# 데이터 요청
response = requests.get(url)

if response.status_code == 200:
    # zip 파일 열기
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # 압축 해제 후 corpCode.xml 읽기
        with z.open('CORPCODE.xml') as xml_file:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            
            # 데이터를 담을 리스트
            corp_list = []

            # XML 데이터 파싱
            for corp in root.findall('list'):
                corp_code = corp.findtext('corp_code')
                corp_name = corp.findtext('corp_name')
                corp_eng_name = corp.findtext('corp_eng_name')
                stock_code = corp.findtext('stock_code')
                modify_date = corp.findtext('modify_date')
                
                corp_list.append({
                    "corp_code": corp_code,
                    "corp_name": corp_name,
                    "corp_eng_name": corp_eng_name,
                    "stock_code": stock_code,
                    "modify_date": modify_date
                })

            # DataFrame으로 변환
            corp_df = pd.DataFrame(corp_list)

            # 결과 출력
            print(corp_df.head())

else:
    print(f"요청 실패: 상태 코드 {response.status_code}")


  corp_code          corp_name  \
0  00434003                 다코   
1  00430964              굿앤엘에스   
2  00388953  크레디피아제이십오차유동화전문회사   
3  00179984             연방건설산업   
4  00420143     브룩스피알아이오토메이션잉크   

                                       corp_eng_name stock_code modify_date  
0                                   Daco corporation               20170630  
1                                 Good & LS Co.,Ltd.               20170630  
2  Credipia 25th Asset Securitization Specialty L...               20170630  
3                                           youn bao               20170630  
4                        BROOKS-PRI Automation, Inc.               20170630  


# 임원 소유 보고서 테스트

In [4]:
import pandas as pd

def test_case_1_no_date():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", limit=5
    )
    assert isinstance(df, pd.DataFrame)
    assert not df.empty
    print("test_case_1_no_date 통과")
    print(df)

def test_case_2_start_only():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", start_date="2025-01-01", limit=5
    )
    assert all(df["rcept_dt"] >= pd.to_datetime("2025-01-01"))
    print("test_case_2_start_only 통과")
    print(df)

def test_case_3_end_only():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", end_date="2025-12-31", limit=5
    )
    assert all(df["rcept_dt"] <= pd.to_datetime("2025-12-31"))
    print("test_case_3_end_only 통과")
    print(df)

def test_case_4_start_and_end():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", start_date="2024-12-01", end_date="2025-12-31", limit=5
    )
    assert all((df["rcept_dt"] >= pd.to_datetime("2024-12-01")) &
               (df["rcept_dt"] <= pd.to_datetime("2025-12-31")))
    print("test_case_4_start_and_end 통과")
    print(df)

def test_case_5_reference_only():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", reference_date="2025-03-22", limit=5
    )
    assert not df.empty
    print("test_case_5_reference_only 통과")
    print(df)

def test_case_6_invalid_stock_code():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="000000", limit=5
    )
    assert df.empty
    print("test_case_6_invalid_stock_code 통과 (빈 결과 확인됨)")

def test_case_7_all_dates():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930",
        start_date="2024-12-01",
        end_date="2025-12-31",
        reference_date="2025-03-12",
        limit=5
    )
    # reference_date는 무시됨
    assert all((df["rcept_dt"] >= pd.to_datetime("2024-12-01")) &
               (df["rcept_dt"] <= pd.to_datetime("2025-12-31")))
    print("test_case_7_all_dates 통과")
    print(df)

def test_case_8_corp_name_with_spaces():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        corp_name=" 삼성 전자 ", reference_date="2025-03-12", limit=5
    )
    assert not df.empty
    print("test_case_8_corp_name_with_spaces 통과")
    print(df)

def test_case_9_both_inputs_provided():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", corp_name="삼성전자", reference_date="2025-03-11", limit=5
    )
    assert not df.empty
    print("test_case_9_both_inputs_provided 통과")
    print(df)

if __name__ == "__main__":
    test_case_1_no_date()
    test_case_2_start_only()
    test_case_3_end_only()
    test_case_4_start_and_end()
    test_case_5_reference_only()
    test_case_6_invalid_stock_code()
    test_case_7_all_dates()
    test_case_8_corp_name_with_spaces()
    test_case_9_both_inputs_provided()


TypeError: DARTExecutiveShareholdingAPI._get_executive_shareholding() missing 1 required positional argument: 'self'

In [3]:
def test_case_7_all_dates():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930",
        start_date="2024-12-01",
        end_date="2025-12-31",
        reference_date="2025-03-12",
        limit=5
    )
    return df

In [6]:
df = test_case_7_all_dates()

# 대량 보유 상황 보고서 API

In [1]:
import sys
import os
import pandas as pd
# 현재 Jupyter Notebook이 실행 중인 디렉토리를 기준으로 상위 경로 추가
sys.path.append(os.path.abspath(".."))

# 모듈 불러오기
from dart_tool.insider_trade_tool import DARTExecutiveShareholdingAPI, DARTMajorStockReportAPI
api = DARTMajorStockReportAPI()

In [22]:

## 기본
def test_majorstock_basic():
    df = api._get_major_stock_reports(corp_name="삼성전자")
    assert not df.empty
    assert "rcept_dt" in df.columns


## 날짜 필터
def test_majorstock_start_date_only():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        start_date="2024-11-01"
    )
    assert not df.empty
    assert df["rcept_dt"].min() >= pd.to_datetime("2024-11-01")


def test_majorstock_end_date_only():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        end_date="2024-12-31"
    )
    assert not df.empty
    assert df["rcept_dt"].max() <= pd.to_datetime("2024-12-31")


def test_majorstock_reference_date():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        reference_date="2024-12-01"
    )
    assert not df.empty
    assert df["rcept_dt"].max() <= pd.to_datetime("2024-12-01")


## 수치 필터
def test_majorstock_min_ratio():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_ratio=20.5
    )
    assert not df.empty
    assert df["stkrt"].astype(float).min() >= 20.5


def test_majorstock_min_ratio_change():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_ratio_change=0.01
    )
    assert not df.empty
    assert df["stkrt_irds"].astype(float).min() >= 0.01


def test_majorstock_min_share_count():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_share_count=1_200_000_000
    )
    assert not df.empty
    assert df["stkqy"].min() >= 1_200_000_000


def test_majorstock_min_share_change():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_share_change=100
    )
    assert not df.empty
    assert df["stkqy_irds"].min() >= 100




def test_majorstock_max_share_change():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        max_share_change=-1
    )
    assert not df.empty
    assert df["stkqy_irds"].max() <= -1



## 날짜 + 수치 필터 조합
def test_majorstock_date_and_share_filter():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        start_date="2021-01-01",
        min_share_change=100
    )
    assert not df.empty
    assert df["stkqy_irds"].min() >= 100



In [ ]:
def main():
    test_majorstock_basic()

    test_majorstock_start_date_only()

    test_majorstock_end_date_only()

    test_majorstock_reference_date()

    test_majorstock_min_ratio()

    test_majorstock_min_ratio_change()

    test_majorstock_min_share_count()

    test_majorstock_min_share_change()

    test_majorstock_max_share_change()

    test_majorstock_date_and_share_filter()
    print("모든 테스트가 통과.")

main()


1
1
1
1
1
1
2
2
3
모든 테스트가 통과되었습니다.


In [10]:
df = api._get_major_stock_reports(corp_name="삼성전자")
print(df[["rcept_dt", "stkqy_irds"]].sort_values(by="stkqy_irds", ascending=False))


     rcept_dt  stkqy_irds
8  2024-10-25       57566
15 2024-05-17       14495
9  2024-10-25        6317
0  2025-03-21      -10682
10 2024-10-25      -11113
3  2025-01-24      -21972
4  2025-01-24      -38791
18 2024-04-19      -47314
12 2024-07-26      -55084
16 2024-05-03      -62823
1  2025-03-07      -77470
5  2024-12-20      -91303
11 2024-10-25     -117240
14 2024-05-24     -205875
6  2024-11-15     -259744
17 2024-04-26     -309948
13 2024-07-19     -347788
7  2024-10-25     -348407
2  2025-02-14    -4963824
19 2024-04-12    -5332668


# 자기 주식 처분 결정 테스트

In [1]:
import sys
import os
import pandas as pd
# 현재 Jupyter Notebook이 실행 중인 디렉토리를 기준으로 상위 경로 추가
sys.path.append(os.path.abspath(".."))

# 모듈 불러오기
from dart_tool.insider_trade_tool import DARTTreasuryStockDispositionDecisionAPI
api = DARTTreasuryStockDispositionDecisionAPI()

In [5]:
# 기본 케이스
def test_treasury_disposal_basic():
    df = api._get_treasury_stock_disposals(
        corp_name="SK하이닉스",
        start_date="2016-01-01",
        end_date="2024-12-31"
    )
    assert not df.empty
    assert "dp_dd" in df.columns
    print("test_treasury_disposal_basic 통과")

# 날짜 범위 안에 데이터가 있는 경우
def test_treasury_disposal_date_range():
    df = api._get_treasury_stock_disposals(
        corp_name="SK하이닉스",
        start_date="2017-01-01",
        end_date="2024-12-31"
    )
    assert not df.empty, "해당 기간에 공시가 없습니다."
    
    # 날짜 문자열 → datetime 변환
    df["dp_dd"] = pd.to_datetime(df["dp_dd"])
    
    assert df["dp_dd"].max() <= pd.to_datetime("2024-12-31")
    assert df["dp_dd"].min() >= pd.to_datetime("2017-01-01")
    print("test_treasury_disposal_date_range 통과")



# 종목코드로 조회
def test_treasury_disposal_with_stock_code():
    df = api._get_treasury_stock_disposals(
        stock_code="000660",
        start_date="2017-01-01",
        end_date="2024-12-31"
    )
    assert not df.empty
    print("test_treasury_disposal_with_stock_code 통과")

# 없는 회사명 테스트 (빈 결과)
def test_treasury_disposal_invalid_corp():
    df = api._get_treasury_stock_disposals(
        corp_name="없는회사",
        start_date="2017-01-01",
        end_date="2024-12-31"
    )
    assert df.empty
    print("test_treasury_disposal_invalid_corp 통과")

# 날짜 미입력 테스트 (필수값 오류)
def test_treasury_disposal_missing_dates():
    df = api._get_treasury_stock_disposals(
        corp_name="SK하이닉스"
    )
    assert df.empty
    print("test_treasury_disposal_missing_dates 통과")

# 전체 실행
def main():
    test_treasury_disposal_basic()
    print("베이직")
    test_treasury_disposal_date_range()
    print("날짜범위")
    test_treasury_disposal_with_stock_code()
    print("티커")
    test_treasury_disposal_invalid_corp()
    print("없는 기업")
    test_treasury_disposal_missing_dates()
    print("모든 테스트")

# 테스트 실행
main()


{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20210429000006', 'corp_cls': 'Y', 'corp_code': '00164779', 'corp_name': 'SK하이닉스', 'od_a_at_t': '6', 'od_a_at_b': '0', 'adt_a_atn': '-', 'cs_iv_bk': 'SK증권(SK Securities.Co., Ltd.)', 'aq_wtn_div_ostk': '44,000,570', 'aq_wtn_div_ostk_rt': '6.04', 'aq_wtn_div_estk': '-', 'aq_wtn_div_estk_rt': '-', 'eaq_ostk': '-', 'eaq_ostk_rt': '-', 'eaq_estk': '-', 'eaq_estk_rt': '-', 'dppln_stk_ostk': '3,618,878', 'dppln_stk_estk': '-', 'dpstk_prc_ostk': '135,000', 'dpstk_prc_estk': '-', 'dppln_prc_ostk': '488,548,530,000', 'dppln_prc_estk': '-', 'dpprpd_bgd': '2021년 04월 29일', 'dpprpd_edd': '2021년 05월 03일', 'dp_pp': '임직원 자기주식 상여지급', 'dp_m_mkt': '-', 'dp_m_ovtm': '-', 'dp_m_otc': '3,618,878', 'dp_m_etc': '-', 'dp_dd': '2021년 04월 28일', 'd1_slodlm_ostk': '-', 'd1_slodlm_estk': '-'}, {'rcept_no': '20220224000002', 'corp_cls': 'Y', 'corp_code': '00164779', 'corp_name': 'SK하이닉스', 'od_a_at_t': '6', 'od_a_at_b': '0', 'adt_a_atn': '-', 'cs_iv_bk': 'SK증권(S